In [1]:
import gmaps
import pandas as pd
from api_keys import gkey
import requests
from pprint import pprint

gmaps.configure(api_key=gkey)

In [2]:
cities_df = pd.read_csv('../WeatherPy/WeatherPy.csv')

cities_df.drop(cities_df.columns[cities_df.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)

cities_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,puerto ayora,75.0,EC,1.587169e+09,88.0,-0.74,-90.35,78.80,6.55
1,hermanus,99.0,ZA,1.587169e+09,86.0,-34.42,19.23,57.99,5.99
2,rikitea,1.0,PF,1.587169e+09,63.0,-23.12,-134.97,77.32,0.96
3,port-gentil,20.0,GA,1.587169e+09,94.0,-0.72,8.78,78.80,6.93
4,new norfolk,58.0,AU,1.587169e+09,79.0,-42.78,147.06,55.99,3.00


In [3]:
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
rating = cities_df["Humidity"].astype(float)
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [4]:
vacations_df = cities_df.loc[(cities_df['Max Temp']>70) & \
                             (cities_df['Max Temp']<80) & \
                             (cities_df['Wind Speed']<10) & \
                             (cities_df['Cloudiness']==0)]

vacations_df["Lat"] = vacations_df["Lat"].astype(str)
vacations_df["Lng"] = vacations_df["Lng"].astype(str)

vacations_df["Lat_Lng"] = vacations_df["Lat"] + ',' + vacations_df["Lng"]

vacations_df

C:\Users\17736\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\17736\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\17736\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Lat_Lng
110,beloha,0.0,MG,1.587169e+09,68.0,-25.17,45.05,71.91,3.56,"-25.17,45.05"
127,bilma,0.0,NE,1.587169e+09,14.0,18.69,12.92,77.90,3.56,"18.69,12.92"
173,san juan,0.0,AR,1.587169e+09,29.0,-31.54,-68.54,75.31,7.05,"-31.54,-68.54"
250,ambovombe,0.0,MG,1.587169e+09,78.0,-25.17,46.08,71.51,8.41,"-25.17,46.08"
258,las margaritas,0.0,MX,1.587169e+09,26.0,16.32,-91.98,74.21,6.49,"16.32,-91.98"
265,satuba,0.0,BR,1.587169e+09,94.0,-9.56,-35.82,77.00,3.36,"-9.56,-35.82"
293,jiwani,0.0,PK,1.587169e+09,76.0,25.05,61.74,78.22,8.19,"25.05,61.74"
382,pathein,0.0,MM,1.587169e+09,84.0,16.78,94.73,75.18,3.78,"16.78,94.73"
420,uthal,0.0,PK,1.587169e+09,74.0,25.81,66.62,74.88,0.67,"25.81,66.62"
430,biu,0.0,NG,1.587169e+09,11.0,10.61,12.19,75.74,5.59,"10.61,12.19"


In [5]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?v=weekly"
params = {"rankby": "distance",
        "type": "lodging",
        "key": gkey,
         }

# use iterrows to iterate through pandas dataframe
for index, row in vacations_df.iterrows():

    # get city info from df
#     location = row[["Lat","Lng"]]
    city = row['City']
    
    # add keyword to params dict
    params['location'] = row['Lat_Lng']

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
 
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel to {city} is {results[0]['name']}.")
        
        vacations_df.loc[index, 'name'] = results[0]['name']
        vacations_df.loc[index, 'address'] = results[0]['vicinity']
        
        
    except (KeyError, IndexError):
          print('skipping')  

skipping
Closest hotel to bilma is Expeditions Ténére voyages.


C:\Users\17736\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\17736\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Closest hotel to san juan is Departamento Hospedaje.
Closest hotel to ambovombe is MONJA Talilisoa.
Closest hotel to las margaritas is Hotel Nolaska.
Closest hotel to satuba is Encontre Sua Viagem - Satuba.
Closest hotel to jiwani is Al Mumtaz Transport Jiwani.
Closest hotel to pathein is The First Hotel.
Closest hotel to uthal is Al Madina Hotal.
Closest hotel to biu is EYN LCC MANDARAGIRAU.


In [6]:
vacations_df


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Lat_Lng,name,address
110,beloha,0.0,MG,1.587169e+09,68.0,-25.17,45.05,71.91,3.56,"-25.17,45.05",NaN,NaN
127,bilma,0.0,NE,1.587169e+09,14.0,18.69,12.92,77.90,3.56,"18.69,12.92",Expeditions Ténére voyages,Bilma
173,san juan,0.0,AR,1.587169e+09,29.0,-31.54,-68.54,75.31,7.05,"-31.54,-68.54",Departamento Hospedaje,"Suipacha 527, J5402FQE"
250,ambovombe,0.0,MG,1.587169e+09,78.0,-25.17,46.08,71.51,8.41,"-25.17,46.08",MONJA Talilisoa,Ambovombe
258,las margaritas,0.0,MX,1.587169e+09,26.0,16.32,-91.98,74.21,6.49,"16.32,-91.98",Hotel Nolaska,"Calle Central Poniente, Centro, Las Margaritas"
265,satuba,0.0,BR,1.587169e+09,94.0,-9.56,-35.82,77.00,3.36,"-9.56,-35.82",Encontre Sua Viagem - Satuba,"Estr. Utinga, 1 - Condomínio Residencial Recan..."
293,jiwani,0.0,PK,1.587169e+09,76.0,25.05,61.74,78.22,8.19,"25.05,61.74",Al Mumtaz Transport Jiwani,Jīwani
382,pathein,0.0,MM,1.587169e+09,84.0,16.78,94.73,75.18,3.78,"16.78,94.73",The First Hotel,"43 Strand Road, Pathein"
420,uthal,0.0,PK,1.587169e+09,74.0,25.81,66.62,74.88,0.67,"25.81,66.62",Al Madina Hotal,"Al Madina Hotal, Uthal"
430,biu,0.0,NG,1.587169e+09,11.0,10.61,12.19,75.74,5.59,"10.61,12.19",EYN LCC MANDARAGIRAU,"LOCAL GOVERNMENT AREA, Biu"


In [7]:
vacations_df['Lat'] = vacations_df['Lat'].astype(float)
vacations_df['Lng'] = vacations_df['Lng'].astype(float)

marker_locations = vacations_df[['Lat', 'Lng']]

# Assign the marker layer to a variable
markers = gmaps.marker_layer(marker_locations)

# Add the layer to the map
fig.add_layer(markers)
fig


C:\Users\17736\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\17736\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Figure(layout=FigureLayout(height='420px'))